## This notebook requires a https://github.com/cocktailpeanut/dalai dalai server running

**Download and unpack dalaipy**

In [ ]:
import os
processing_path = "../../data/question_answering/"
os.makedirs(processing_path, exist_ok=True)

!wget https://github.com/wastella/dalaipy/archive/refs/heads/main.zip -P {processing_path}
unzip_path_extractor = processing_path + "main.zip"
!unzip {unzip_path_extractor} -d {processing_path}

**Move main file to current directory and import it**

In [1]:
!cp ../../data/question_answering/dalaipy-main/src/main.py daiaipy.py
from  daiaipy import Dalai
model = Dalai()

**function used for question generation**

In [25]:
import time 
def get_question(request):
    # used for testing
    # request = request[:40]
    # used for testing

    request_sentence ="The generated question should be answerable only by the information provided in the sentence. Fill the last field:\nAnswer: The capital of France is Paris. Generated question: What is the capital of France?\nAnswer: Green Dragon: The Green Dragon was a British pub. Generated question:What was the Green Dragon?\nAnswer: The movie was directed by Steven Spielberg. Generated question: Who directed the movie?\nAnswer: The temperature is 25 degrees Celsius. Generated question: What is the temperature in Celsius?\nAnswer: "
    request_sentence = request_sentence + request + " Generated question:"
    #request_sentence = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\n>Ask a question that can be answered by the following sentences and tell me what the answer in the sentence is.:\n### Input:\n>"
    #request_sentence = request_sentence + request + "\n### Response:"

    request_dict = model.generate_request(request_sentence, "alpaca.7B")
    str = model.generate(request_dict)['response'].replace("\r", "")[len(request_sentence):-7]
    time.sleep(4)
    return str.split("?")[0] + "?"

In [ ]:
get_question("Lobalug: The Lobalug was an aquatic magical beast found at the bottom of the North Sea.")

In [ ]:
import pandas as pd
df = pd.read_pickle('../../data/dataframes/harrypotter_pages_current_cleaned.pickle')
df

In [11]:
from transformers import T5Config, T5ForConditionalGeneration, T5Tokenizer

model_name = "allenai/t5-small-squad2-question-generation"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    return output[0]

In [ ]:
df['questions'] = df['text'].apply(run_model)
df

In [ ]:

import numpy as np
pd.set_option('display.max_colwidth', 300)
for i in np.random.randint(len(df), size= 10):
    print(str(i) + " - " +  ": ")
    print(df.iloc[i]["text"])
    print("-----------question----------------")
    print(df.iloc[i]["questions"])
    print("------------------------------------------")

